In [13]:
# Creating DB Connection
from sqlalchemy import create_engine

engine = create_engine("sqlite:///example.db", echo=True)

In [14]:
# Define a table:

from sqlalchemy import Column, Integer, String, Sequence
from sqlalchemy.orm import declarative_base

Base = declarative_base()


class User(Base):
    __tablename__ = "users"

    id = Column(Integer, Sequence("user_id_seq"), primary_key=True)
    name = Column(String(50))
    email = Column(String(50))

In [15]:
# Create the table in the database:
Base.metadata.create_all(engine)

2023-04-01 00:53:06,650 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-01 00:53:06,651 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-04-01 00:53:06,653 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-01 00:53:06,655 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
# Add a record to the table:

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

new_user = User(name="John Doe", email="jdoe@example.com")
session.add(new_user)
session.commit()

2023-04-01 00:53:06,712 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-01 00:53:06,715 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2023-04-01 00:53:06,716 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('John Doe', 'jdoe@example.com')
2023-04-01 00:53:06,721 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
# Query for records:
users = session.query(User).all()
for user in users:
    print(user.name, user.email)

2023-04-01 00:53:06,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-01 00:53:06,848 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users
2023-04-01 00:53:06,849 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
John Doe jdoe@example.com


In [18]:
# Filter records based on a condition:
john = session.query(User).filter_by(name="John Doe").first()

2023-04-01 00:53:06,952 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-04-01 00:53:06,955 INFO sqlalchemy.engine.Engine [generated in 0.00231s] ('John Doe', 1, 0)


In [19]:
# Update a record:
john.email = "john.doe@example.com"
session.commit()

2023-04-01 00:53:07,015 INFO sqlalchemy.engine.Engine UPDATE users SET email=? WHERE users.id = ?
2023-04-01 00:53:07,015 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('john.doe@example.com', 1)
2023-04-01 00:53:07,018 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
# Delete a record:

session.delete(john)
session.commit()

2023-04-01 00:53:07,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-01 00:53:07,109 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
2023-04-01 00:53:07,110 INFO sqlalchemy.engine.Engine [generated in 0.00094s] (1,)
2023-04-01 00:53:07,112 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2023-04-01 00:53:07,113 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (1,)
2023-04-01 00:53:07,114 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
# Execute a raw SQL statement:
from sqlalchemy import text

query = text("SELECT * FROM users WHERE name = 'John Doe';")

result = session.execute(query)
for row in result:
    print(row)

2023-04-01 00:57:32,319 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE name = 'John Doe';
2023-04-01 00:57:32,319 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()


In [31]:
# Insert a new user into the database using a raw SQL statement:
query = text("INSERT INTO users (name, email) VALUES ('Jane Doe', 'jane@example.com');")

session.execute(query)
session.commit()

2023-04-01 00:59:05,206 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES ('Jane Doe', 'jane@example.com');
2023-04-01 00:59:05,207 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ()
2023-04-01 00:59:05,211 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
# To visualize SQLite in Jupyter Notebook, you can use the ipython-sql library.

! poetry add ipython-sql

Using version ^0.5.0 for ipython-sql

Updating dependencies
Resolving dependencies...

Writing lock file

Package operations: 3 installs, 0 updates, 0 removals

  • Installing prettytable (3.6.0)
  • Installing sqlparse (0.4.3)
  • Installing ipython-sql (0.5.0)


In [33]:
%load_ext sql

In [34]:
%sql sqlite:///example.db

In [38]:
%sql select * from users

 * sqlite:///example.db
Done.


id,name,email
1,Jane Doe,jane@example.com


In [39]:
# create 10 user records and add them to the session
for i in range(10):
    user = User(name=f"User {i}", email=f"user{i}@example.com")
    session.add(user)

In [40]:
%sql select * from users

 * sqlite:///example.db
Done.


id,name,email
1,Jane Doe,jane@example.com


In [41]:
# commit the changes to the database
session.commit()

2023-04-01 01:06:05,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-01 01:06:05,923 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?) RETURNING id
2023-04-01 01:06:05,923 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues)] ('User 0', 'user0@example.com', 'User 1', 'user1@example.com', 'User 2', 'user2@example.com', 'User 3', 'user3@example.com', 'User 4', 'user4@example.com', 'User 5', 'user5@example.com', 'User 6', 'user6@example.com', 'User 7', 'user7@example.com', 'User 8', 'user8@example.com', 'User 9', 'user9@example.com')
2023-04-01 01:06:05,928 INFO sqlalchemy.engine.Engine COMMIT


In [42]:
%sql select * from users

 * sqlite:///example.db
Done.


id,name,email
1,Jane Doe,jane@example.com
2,User 0,user0@example.com
3,User 1,user1@example.com
4,User 2,user2@example.com
5,User 3,user3@example.com
6,User 4,user4@example.com
7,User 5,user5@example.com
8,User 6,user6@example.com
9,User 7,user7@example.com
10,User 8,user8@example.com
